In [1]:
# Import necessary libraries for data processing and API calls
from pyspark.sql.functions import col, isnull, when
# Import necessary libraries for data processing and API calls
from pyspark.sql.types import TimestampType
# Import necessary libraries for data processing and API calls
from datetime import date, timedelta


StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 3, Finished, Available, Finished)

In [2]:

# Remove this before running Data Factory Pipeline
start_date = date. today() - timedelta(7)


StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 4, Finished, Available, Finished)

In [3]:

# Load the JSON data into a Spark DataFrame
# Load data into a Spark DataFrame for processing
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")

StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 5, Finished, Available, Finished)

In [4]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 6, Finished, Available, Finished)

In [5]:
# Validate data: Check for missing or null values
df = (
    df
# Apply conditional transformations to clean and filter the dataset
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
# Apply conditional transformations to clean and filter the dataset
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
# Apply conditional transformations to clean and filter the dataset
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 7, Finished, Available, Finished)

In [6]:

# Convert 'time' and 'updated' to timestamp
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)


StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 8, Finished, Available, Finished)

In [7]:
# Append to the stage2 table
# Save the transformed data into a Delta table for querying
df.write.mode('append').saveAsTable('earthquake_events_stage2')

StatementMeta(, 26d625f0-25ed-4009-b9e4-e3adacd8bc1d, 9, Finished, Available, Finished)